In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from statsmodels.tools.eval_measures import rmse

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# sklearn.__version__
# pip install scikit-learn==0.24.1

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout

import warnings
warnings.filterwarnings('ignore')
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)


sns.set()
# https://archive.ics.uci.edu/ml/datasets/Power+consumption+of+Tetouan+city#

In [3]:
# Preparing data set
data = pd.read_csv("assets/Tetuan_City_power_consumption.csv") 
data.columns = data.columns.str.replace(' ','_')
data.DateTime = pd.to_datetime(data.DateTime)
data.set_index('DateTime', inplace=True)
# Zone 1
data_zone1 = data[['Zone_1_Power_Consumption']]

# Changing frequency to daily
data_zone1 = data_zone1.resample('D').sum()

In [3]:
df_rnn_cv = data_zone1[['Zone_1_Power_Consumption']]

num = 7

# Cross Validation on RNN model


<img src="assets/time_series_cv_plot.jpg"  align="center"/>

In [4]:
# RNN Architecture

n_features = 1
n_input = num

model_rnn = Sequential()

model_rnn.add(LSTM(units=170, return_sequences=True, activation='relu' , input_shape=(n_input, n_features)))
model_rnn.add(Dropout(0.1))

model_rnn.add(LSTM(units=160, return_sequences=True, activation='relu'))
model_rnn.add(Dropout(0.1))

model_rnn.add(LSTM(units=150, return_sequences=True, activation='relu'))
model_rnn.add(Dropout(0.1))

model_rnn.add(LSTM(units=140, return_sequences=True, activation='relu'))
model_rnn.add(Dropout(0.1))

model_rnn.add(LSTM(units=130, return_sequences=True, activation='relu'))
model_rnn.add(Dropout(0.1))

model_rnn.add(LSTM(units=120, activation='relu'))
model_rnn.add(Dropout(0.1))

model_rnn.add(Dense(units=1))

In [5]:
model_rnn.compile(optimizer='adam', loss='mse')
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 170)            116960    
                                                                 
 dropout (Dropout)           (None, 7, 170)            0         
                                                                 
 lstm_1 (LSTM)               (None, 7, 160)            211840    
                                                                 
 dropout_1 (Dropout)         (None, 7, 160)            0         
                                                                 
 lstm_2 (LSTM)               (None, 7, 150)            186600    
                                                                 
 dropout_2 (Dropout)         (None, 7, 150)            0         
                                                                 
 lstm_3 (LSTM)               (None, 7, 140)            1

In [6]:
# Cross Validation

n_input = 7
epochs = 200

# Intializing Metrics variables

mse = []
rmse = []
mae = []
mape = []
r_2 = []

# 5 fold split, test size equal to 7

tscv = TimeSeriesSplit(n_splits = 5, test_size=7)
for train_index, test_index in tscv.split(df_rnn_cv):
    train_cv, test_cv = df_rnn_cv.iloc[train_index], df_rnn_cv.iloc[test_index]
    
    
    # Scale our data

    scaler = MinMaxScaler()
    scaler.fit(train_cv) # Fit only to training
    scaled_train_cv = scaler.transform(train_cv)
    scaled_test_cv = scaler.transform(test_cv)
    

    # Generator
    
    ts_generator_cv = TimeseriesGenerator(scaled_train_cv, scaled_train_cv, length=n_input, batch_size=1)
    
    
    # Fitting model
    
    model_rnn.fit_generator(generator = ts_generator_cv, epochs=epochs, verbose=0)
    
    
    # Prediction on test dataset
    
    rnn_predictions_cv= []
    current_data_cv = scaled_train_cv[-(n_input):].reshape((1, n_input,1))
    for i in range(n_input):
        current_prediction_cv = model_rnn.predict(current_data_cv,verbose=0)[0]
        rnn_predictions_cv.append(current_prediction_cv)
        current_data_cv= np.append(current_data_cv[:,1:,:],[[current_prediction_cv]],axis=1)

        
    # Inversing scaled predictions
    
    predictions = scaler.inverse_transform(rnn_predictions_cv)
    true_values = test_cv.Zone_1_Power_Consumption
    
    
    # Accuracy metrics
    
    mse.append(mean_squared_error(true_values, predictions))
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
    mae.append(mean_absolute_error(true_values, predictions))
    mape.append(mean_absolute_percentage_error(true_values, predictions))
    r_2.append(r2_score(y_true=true_values, y_pred=predictions))

In [14]:
# Comparing different metrics and mean of the metrics in cross validation
metrics = pd.DataFrame()
#metrics.loc[:,"Mean Squared Erorr"] = [round(x) for x in mse]
metrics.loc[:,"Root Mean Squared Erorr"]        = [round(x) for x in rmse]
metrics.loc[:,"Mean absolout erorr"]            = [round(x) for x in mae]
metrics.loc[:,"Mean absolout percentage erorr"] = [100*round(x,3) for x in mape]
metrics.loc['mean'] = [round(num,2) for num in metrics.mean()]
metrics

,Root Mean Squared Erorr,Mean absolout erorr,Mean absolout percentage erorr
0,160846.0,125460.0,3.10
1,123488.0,91619.0,2.20
2,96983.0,83960.0,2.00
3,73632.0,56745.0,1.40
4,70335.0,63010.0,1.50
mean,105056.8,84158.8,2.04


# Cross Validation on Arima model


In [20]:
# Intializing Metrics variables

rmse_arima = []
mse_arima = []
mae_arima = []
mape_arima = []
r_2_arima = []


# 5 fold split, test size equal to 7

tscv = TimeSeriesSplit(n_splits = 5, test_size=7)
for train_index, test_index in tscv.split(df_rnn_cv):
    cv_train, cv_test = df_rnn_cv.iloc[train_index], df_rnn_cv.iloc[test_index]
    
    # Fitting
    model = SARIMAX(cv_train, order= (3,0,4), seasonal_order= (1,1,1,7)).fit()
    
    # Prediction
    predictions = model.predict(cv_test.index.values[0], cv_test.index.values[-1])
    true_values = cv_test.values
    rmse_arima.append(np.sqrt(mean_squared_error(true_values, predictions)))
    mse_arima.append(mean_squared_error(true_values, predictions))
    mae_arima.append(mean_absolute_error(true_values, predictions))
    mape_arima.append(mean_absolute_percentage_error(true_values, predictions))
    r_2_arima.append(r2_score(y_true=true_values, y_pred=predictions))
    


In [21]:
# Comparing different metrics and mean of the metrics in cross validation
metrics_arima = pd.DataFrame()
#metrics_arima.loc[:,"Mean Squared Erorr"] = [round(num) for num in mse_arima]
metrics_arima.loc[:,"Root Mean Squared Erorr"] = [round(num) for num in rmse_arima]
metrics_arima.loc[:,"Mean absolout erorr"] = [round(num) for num in mae_arima]
metrics_arima.loc[:,"Mean absolout percentage erorr"] = [100*round(num,3) for num in mape_arima]
metrics_arima.loc['mean'] = [round(num,2) for num in metrics_arima.mean()]
metrics_arima

,Root Mean Squared Erorr,Mean absolout erorr,Mean absolout percentage erorr
0,138128.0,118436.0,2.90
1,96240.0,72746.0,1.80
2,57902.0,47631.0,1.10
3,38357.0,23422.0,0.60
4,65347.0,59536.0,1.40
mean,79194.8,64354.2,1.56


# Arima Model Plot
<img src="assets/Arima_future_prediction_plot.JPG"  align="center"/>
# RNN Model Plot.JPG
<img src="assets/rnn_future_prediction_plot.JPG"  align="center"/>
